In [ ]:
import json
import requests

from article_scraper import NewsScraper


# Load keys
with open('../../keys.json') as f:
    keys = json.load(f)

NEWS_API_KEY = keys["NEWSAPI_KEY"]
POST_NAME = keys['POST_USERNAME']
POST_PASS = keys["POST_PASSWORD"]

# Pull article info from NewsAPI
url = 'https://newsapi.org/v2/everything?'
url = 'https://eventregistry.org/api/v1/article/getArticles'
parameters = {
    'q': 'election', # query phrase
    'from': '2024-11-15',
    'to': '2024-11-21',
    'apiKey': NEWS_API_KEY,
    'articlesPage':1,
    'domains':'foxnews.com'
}

parameters = {
  "action": "getArticles",
  "keyword": "Tesla Inc",
  "sourceLocationUri": [
    "http://en.wikipedia.org/wiki/United_States",
    "http://en.wikipedia.org/wiki/Canada",
    "http://en.wikipedia.org/wiki/United_Kingdom"
  ],
  "ignoreSourceGroupUri": "paywall/paywalled_sources",
  "articlesPage": 1,
  "articlesCount": 100,
  "articlesSortBy": "date",
  "articlesSortByAsc": False,
  "dataType": [
    "news",
    "pr"
  ],
  "forceMaxDataTimeWindow": 31,
  "resultType": "articles",
  "apiKey": NEWS_API_KEY
}

response = requests.get(url, params=parameters)

In [72]:
response

<Response [401]>

In [71]:
[article['url'] for article in response.json()['articles']]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [56]:
[article['url'] for article in response.json()['articles']]

['https://www.foxnews.com/politics/pennsylvania-democrats-openly-admit-counting-illegal-ballots-mccormick-casey-race',
 'https://www.foxnews.com/politics/minnesota-election-judge-faces-felony-charges-over-accepting-unregistered-votes',
 'https://www.foxnews.com/media/joe-rogan-says-many-entertainment-industry-had-privately-thanked-me-endorsing-trump',
 'https://www.foxnews.com/lifestyle/nine-countries-offering-ancestry-based-citizenship-americans-looking-leave-us-after-election',
 'https://www.foxnews.com/media/mom-living-alopecia-reacts-liberal-women-shaving-heads-unattractive-donald-trumps-win',
 'https://www.foxnews.com/media/veteran-pollster-announces-shes-moving-to-other-ventures-after-backlash-bombshell-iowa-poll',
 'https://www.foxnews.com/politics/ranked-choice-voting-dealt-blow-voters-rejected-numerous-states',
 'https://www.foxnews.com/media/democrats-privately-gripe-about-pelosis-damaging-post-election-comments-she-needs-take-seat',
 'https://www.foxnews.com/media/adam-schif